## Constant Tensors

In [1]:
import tensorflow as tf

In [2]:
x = tf.constant([[5, 2], [1, 3]])
print(x)

tf.Tensor(
[[5 2]
 [1 3]], shape=(2, 2), dtype=int32)


In [3]:
x.numpy()

array([[5, 2],
       [1, 3]], dtype=int32)

In [4]:
x.dtype

tf.int32

In [5]:
x.shape

TensorShape([2, 2])

In [6]:
print(tf.ones(shape=(2, 1)))
print(tf.zeros(shape=(2, 1)))

tf.Tensor(
[[1.]
 [1.]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0.]
 [0.]], shape=(2, 1), dtype=float32)


In [7]:
x = tf.random.normal(shape=(2, 2), mean=0.0, stddev=1.0)
x

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[0.07273506, 1.347767  ],
       [0.6886016 , 1.2003833 ]], dtype=float32)>

In [8]:
x = tf.random.uniform(shape=(2, 2), minval=0, maxval=10, dtype="int32")
x

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[0, 2],
       [0, 5]], dtype=int32)>

## Variable

In [9]:
a = tf.Variable(5)
a

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=5>

In [10]:
a.assign(10)
a

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=10>

In [11]:
a.assign_add(3)
a

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=13>

## Math & operations...

In [12]:
a = tf.random.normal(shape=(2, 2))
b = tf.random.normal(shape=(2, 2))
print(a)
print(b)

tf.Tensor(
[[-0.18800011 -0.47738707]
 [ 1.2664824  -1.3369614 ]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[0.42796123 0.09513063]
 [0.741352   1.3965565 ]], shape=(2, 2), dtype=float32)


In [13]:
c = a + b
c

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 0.23996112, -0.38225645],
       [ 2.0078344 ,  0.05959511]], dtype=float32)>

In [14]:
d = tf.square(c)
d

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[5.7581339e-02, 1.4612000e-01],
       [4.0313993e+00, 3.5515770e-03]], dtype=float32)>

In [15]:
e = tf.exp(d)
e

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 1.0592715,  1.157335 ],
       [56.33969  ,  1.0035579]], dtype=float32)>

In [16]:
tf.matmul(a, b)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[-0.43436864, -0.6845826 ],
       [-0.44915366, -1.7466608 ]], dtype=float32)>

In [17]:
d = tf.constant([[-2, 6, 12, 0 , -5], [12, -5, 3, 2 , 0]], dtype='float32')

In [18]:
tf.nn.relu(d)

<tf.Tensor: shape=(2, 5), dtype=float32, numpy=
array([[ 0.,  6., 12.,  0.,  0.],
       [12.,  0.,  3.,  2.,  0.]], dtype=float32)>

In [19]:
tf.nn.softmax(d)

<tf.Tensor: shape=(2, 5), dtype=float32, numpy=
array([[8.2946684e-07, 2.4726060e-03, 9.9752039e-01, 6.1289770e-06,
        4.1296722e-08],
       [9.9982506e-01, 4.1392134e-08, 1.2338821e-04, 4.5391989e-05,
        6.1431374e-06]], dtype=float32)>

## Model subclassing to create a new Layer

In [20]:
from tensorflow import keras

class Linear(keras.layers.Layer):
    """y = w.x + b"""

    def __init__(self, units=100, input_dim=784):
        super().__init__()
        self.w = self.add_weight(
            shape=(input_dim, units), initializer="random_normal", trainable=True
        )
        self.b = self.add_weight(shape=(units,), initializer="zeros", trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


In [21]:
# Instantiate our layer.
linear_layer = Linear(units=100, input_dim=25)

# The layer can be treated as a function.
# Here we call it on some data.
y = linear_layer(tf.ones((10, 25)))
assert y.shape == (10, 100)

In [22]:
len(linear_layer.weights)

2

In [23]:
assert linear_layer.weights == [linear_layer.w, linear_layer.b]

### lets create a sequential Model with out custom Layer:

In [24]:
model = keras.Sequential()
model.add(Linear(100, 784))
model.add(keras.layers.Activation('relu'))
model.add(Linear(200, 100))
model.add(keras.layers.Activation('relu'))
model.add(Linear(10, 200))
model.add(keras.layers.Activation('softmax'))

In [25]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.reshape(-1, 784).astype('float32') / 255.0
x_test = x_test.reshape(-1, 784).astype('float32') / 255.0

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [26]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=1, validation_data=(x_test, y_test))

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.8624 - loss: 0.4602 - val_accuracy: 0.9562 - val_loss: 0.1381


### do not repeat input_dim!
#### Layer weight creation in build(input_shape)


In [27]:
class Linear(keras.layers.Layer):
    """y = w.x + b"""

    def __init__(self, units=32):
        super().__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


# Instantiate our layer.
linear_layer = Linear(4)

# This will also call `build(input_shape)` and create the weights.
y = linear_layer(tf.ones((10, 784)))


In [28]:
model = keras.Sequential()
model.add(keras.Input(shape=(784,)))
model.add(Linear(100))
model.add(keras.layers.Activation('relu'))
model.add(Linear(200))
model.add(keras.layers.Activation('relu'))
model.add(Linear(10))
model.add(keras.layers.Activation('softmax'))

In [29]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

Epoch 1/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.8695 - loss: 0.4550 - val_accuracy: 0.9608 - val_loss: 0.1288
Epoch 2/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9651 - loss: 0.1111 - val_accuracy: 0.9656 - val_loss: 0.1065


### Lets give activation as an Input

In [30]:
class FC(keras.layers.Layer):
    """y = Activation(w.x + b)"""

    def __init__(self, units=32, activation='linear'):
        super().__init__()
        self.units = units
        if activation=='linear':
          self.activation = lambda x:x
        elif activation=='relu':
          self.activation = tf.nn.relu
        elif activation=='tanh':
          self.activation = tf.nn.tanh
        elif activation=='sigmoid':
          self.activation = tf.nn.sigmoid
        elif activation=='softmax':
          self.activation = tf.nn.softmax

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return self.activation (tf.matmul(inputs, self.w) + self.b)


# Instantiate our layer.
linear_layer = FC(25)

# This will also call `build(input_shape)` and create the weights.
y = linear_layer(tf.ones((10, 784)))


In [31]:
model = keras.Sequential()
model.add(keras.Input(shape=(784,)))
model.add(FC(100, activation='relu'))
model.add(FC(200, activation='relu'))
model.add(FC(10, activation='softmax'))

In [32]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

Epoch 1/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.8608 - loss: 0.4627 - val_accuracy: 0.9591 - val_loss: 0.1302
Epoch 2/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9662 - loss: 0.1134 - val_accuracy: 0.9727 - val_loss: 0.0892
